In [1]:
import sqlite3
import pandas as pd
import altair as alt
import numpy as np

In [2]:
conn = sqlite3.connect('G:\\내 드라이브\\sync DeskTop-LapTop (2024.08)\\files\\data\\상권분석.db')

In [3]:
c = conn.cursor()

In [1]:
# c.execute('''
#           select *  from Market_info
#           ''')

# for row in c.fetchall():
#     print(row)

In [17]:
query = 'select *  from Market_info limit 1'

column_check = pd.read_sql_query(query, conn)
column_check

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,MA0101202210A0084547,금강산노래광장,None,I2,음식,I211,주점,I21101,일반 유흥 주점,I56211,...,4.217010e+24,파크장,강원특별자치도 동해시 송정로 11,240806,25789,None,지,None,129.127525,37.495265


In [22]:
column_check.columns

Index(['상가업소번호', '상호명', '지점명', '상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드',
       '상권업종중분류명', '상권업종소분류코드', '상권업종소분류명', '표준산업분류코드', '표준산업분류명', '시도코드',
       '시도명', '시군구코드', '시군구명', '행정동코드', '행정동명', '법정동코드', '법정동명', '지번코드',
       '대지구분코드', '대지구분명', '지번본번지', '지번부번지', '지번주소', '도로명코드', '도로명', '건물본번지',
       '건물부번지', '건물관리번호', '건물명', '도로명주소', '구우편번호', '신우편번호', '동정보', '층정보',
       '호정보', '경도', '위도'],
      dtype='object')

In [4]:
## 각 지역의 인구수와 상가 수를 비교합니다.
query = '''
    SELECT 상권업종대분류명
         , 시도명
         , 시군구명
         , 행정동명
         , 법정동명
         , count(*) as cnt
         from Market_info
         GROUP BY 상권업종대분류명
         , 시도명
         , 시군구명
         , 행정동명
         , 법정동명
'''

df_market_cnt_by_region = pd.read_sql_query(query, conn)
df_market_cnt_by_region.head()

,상권업종대분류명,시도명,시군구명,행정동명,법정동명,cnt
0,과학·기술,강원특별자치도,강릉시,강남동,노암동,20
1,과학·기술,강원특별자치도,강릉시,강남동,월호평동,2
2,과학·기술,강원특별자치도,강릉시,강남동,유산동,4
3,과학·기술,강원특별자치도,강릉시,강남동,장현동,3
4,과학·기술,강원특별자치도,강릉시,강동면,강동면,11


In [28]:
df_market_cnt = df_market_cnt_by_region.groupby(['상권업종대분류명','시도명'])['cnt'].sum().reset_index()
df_market_cnt

,상권업종대분류명,시도명,cnt
0,과학·기술,강원특별자치도,5196
1,과학·기술,경기도,61141
2,교육,강원특별자치도,3114
3,교육,경기도,28261
4,보건의료,강원특별자치도,583
5,보건의료,경기도,6318
6,부동산,강원특별자치도,2306
7,부동산,경기도,26451
8,소매,강원특별자치도,22468
9,소매,경기도,128892


In [37]:
(
    alt.
    Chart(df_market_cnt).
    encode(
        y = alt.X('상권업종대분류명', sort = '-x'),
        x = 'cnt',
        row = '시도명'
    ).
    mark_bar()
)

alt.Chart(...)

In [18]:
query = '''
    select 시군구명
	 , count(*) as cnt
	from Market_info
	where 시도명 = '경기도'
	  and 상호명 like '공차%'
	GROUP BY 시군구명
	order by 2 desc;
    '''

df_gongcha = pd.read_sql_query(query, conn)

In [20]:
## 지역별 공차
total_cnt = np.sum(df_gongcha['cnt'])

df_gongcha['ratio'] = df_gongcha['cnt'].apply(lambda x: x / total_cnt)
df_gongcha.head()


,시군구명,cnt,ratio
0,화성시,13,0.080745
1,성남시 분당구,10,0.062112
2,평택시,9,0.055901
3,파주시,9,0.055901
4,수원시 영통구,8,0.049689


In [21]:
(
    alt.
    Chart(df_gongcha).
    encode(
        x = alt.X('시군구명', sort = '-y'),
        y ='cnt',
        color = alt.condition(
            alt.FieldOneOfPredicate(field='시군구명', oneOf=['고양시 덕양구', '고양시 일산동구', '고양시 일산서구']),
            alt.value('red'),
            alt.value('lightgray')
        )
    ).
    mark_bar()
)


alt.Chart(...)